In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from google.colab import files
import io

songsuploaded = files.upload()
historyuploaded = files.upload()

# load the dataset of songs and their features
songs_df = pd.read_csv(io.BytesIO(songsuploaded['songs.csv']))

# load the user's listening history
listening_history_df = pd.read_csv(io.BytesIO(historyuploaded['user_listening_history.csv']))

# create a new column in the songs dataframe that indicates whether the song has been listened to by the user (1) or not (0)
songs_df["is_listened"] = songs_df["song_id"].isin(listening_history_df["song_id"]).astype(int)

# select the features to use for the model
features = ["artist", "genre", "year", "duration"]

# select the target variable
target = "is_listened"

# create a label encoder for the string features
label_encoder = LabelEncoder()

# transform the string features using the label encoder
songs_df["artist"] = label_encoder.fit_transform(songs_df["artist"])
songs_df["genre"] = label_encoder.fit_transform(songs_df["genre"])

# create a training and test set
X_train, X_test, y_train, y_test = train_test_split(songs_df[features], songs_df[target], test_size=0.2)

# train a random forest classifier on the training set
model = RandomForestClassifier()
model.fit(X_train, y_train)

# score the model on the test set
score = model.score(X_test, y_test)

print("Model accuracy: {:.2f}%".format(score * 100))

# use the model to predict which songs the user is likely to listen to
predictions = model.predict(songs_df[features])

# create a new column in the songs dataframe with the model's predictions
songs_df["prediction"] = predictions

# recommend the top 10 songs that the model predicts the user will listen to
recommended_songs = songs_df.sort_values("prediction", ascending=False).head(10)["song_name"].tolist()

print("Based on your listening history, we recommend the following songs:")
print(recommended_songs)

In [ ]:
from flask import Flask, request, render_template, jsonify
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from google.colab import files
import io

songsuploaded = files.upload()
historyuploaded = files.upload()

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    # get the uploaded files
    songs_df = pd.read_csv(io.BytesIO(songsuploaded['songs.csv']))
    listening_history_df = pd.read_csv(io.BytesIO(historyuploaded['user_listening_history.csv']))

    # create a new column in the songs dataframe that indicates whether the song has been listened to by the user (1) or not (0)
    songs_df["is_listened"] = songs_df["song_id"].isin(listening_history_df["song_id"]).astype(int)

    # select the features to use for the model
    features = ["artist", "genre", "year", "duration"]

    # select the target variable
    target = "is_listened"

    # create a label encoder for the string features
    label_encoder = LabelEncoder()

    # transform the string features using the label encoder
    songs_df["artist"] = label_encoder.fit_transform(songs_df["artist"])
    songs_df["genre"] = label_encoder.fit_transform(songs_df["genre"])

    # create a training and test set
    X_train, X_test, y_train, y_test = train_test_split(songs_df[features], songs_df[target], test_size=0.2)

    # train a random forest classifier on the training set
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    # score the model on the test set
    score = model.score(X_test, y_test)

    # use the model to predict which songs the user is likely to listen to
    predictions = model.predict(songs_df[features])

    # create a new column in the songs dataframe with the model's predictions
    songs_df["prediction"] = predictions

    # recommend the top 10 songs that the model predicts the user will listen to
    recommended_songs = songs_df.sort_values("prediction", ascending=False).head(10)["song_name"].tolist()

    # return the recommended songs as a JSON response
    return jsonify(recommended_songs)

if __name__ == '__main__':
    app.run()